# Neural Collaborative Filtering Model

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('../data/random_100k_sample.csv', sep=';')

In [3]:
user_id_map = { user_id: idx for idx, user_id in enumerate(data['CustomerID'].unique()) }
movie_id_map = { movie_id: idx for idx, movie_id in enumerate(data['MovieID'].unique()) }

data['user'] = data['CustomerID'].map(user_id_map)
data['movie'] = data['MovieID'].map(movie_id_map)

In [4]:
x = data[['user', 'movie']].values
y = data['Rate'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
num_users = len(user_id_map)
num_movies = len(movie_id_map)
embedding_size = 50

## User and movie embedding
Here, users and movies are represented in a vector dimension of a certain size. Then, the embedding vector for each user or movie is made suitable as input to dense layers (in short, flat). Dense layers work with one-dimensional arrays. Flattening is a must, the model cannot send this data to dense layers.

In [6]:
user_input = tf.keras.layers.Input(shape=(1,), name='user_input')
movie_input = tf.keras.layers.Input(shape=(1,), name='movie_input')

user_embedding = tf.keras.layers.Embedding(input_dim=num_users, output_dim=embedding_size, name='user_embedding')(user_input)
movie_embedding = tf.keras.layers.Embedding(input_dim=num_movies, output_dim=embedding_size, name='movie_embedding')(movie_input)

user_flatten = tf.keras.layers.Flatten()(user_embedding)
movie_flatten = tf.keras.layers.Flatten()(movie_embedding)

## Concatenation
Here, the user and film vectors prepared for the dense layers are combined.

In [7]:
concat = tf.keras.layers.Concatenate()([user_flatten, movie_flatten])

## Feature Learning with Fully Connected Layers
Fully connected layers are created here. In the first layer, the number of neurons is 128. In other words, 128 different features are tried to be learned. The ReLu activation function brings the negatives closer to zero and leaves the positives as they are. Non-linear relationships are learned. Concat is the combination of the user and movie vectors we created above. Our first layer starts the learning process from here. In the 2nd layer; it adds 64 more features to the 128 features learned in the first layer. The dense1 at the end shows that it uses the features of the previous layer. In the 3rd layer, the learning process is completed by taking the output of the 2nd layer.

In [8]:
dense1 = tf.keras.layers.Dense(64, activation='relu', name='dense1')(concat)
dense2 = tf.keras.layers.Dense(32, activation='relu', name='dense2')(dense1)
dense3 = tf.keras.layers.Dense(32, activation='relu', name='dense3')(dense2)
output = tf.keras.layers.Dense(1, activation='linear', name='output')(dense3)

Here the model is created. The inputs are the user and movie inputs, which are the information the model will receive, and the output is the user's predicted score to be predicted. Adam optimization algorithm was used. The model's error was measured with MSE. (RMSE will be calculated by taking the square of all values ​​while calculating the overall accuracy.)


In [9]:
model = tf.keras.Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 movie_input (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 user_embedding (Embedding)  (None, 1, 50)                3863900   ['user_input[0][0]']          
                                                                                                  
 movie_embedding (Embedding  (None, 1, 50)                465050    ['movie_input[0][0]']         
 )                                                                                            

The training of the model starts here. x_train is our training set. [x_train[:, 0], x_train[:, 1]] Here the movie and user columns are separated from each other. These two columns are the input data from which the model will receive user and movie information. y_train contains the target data that the model will try to predict. In this case, the target value will be the score that the user will give to the movie. validation_data is the validation step. The performance of the model is measured with data that it has not seen before in the part where it is still learning. Epoch is the number of times the model makes a full transformation on the training data set. Since 10 epochs are specified here, the model will work on the training data 10 times. Batch size is actually the number of iterations. In other words, it is the number of data samples to be presented to the model. verbose is how much detailed information will be given in the output of the training process.

In [10]:
history = model.fit(
    [x_train[:, 0], x_train[:, 1]], y_train,
    validation_data=([x_test[:, 0], x_test[:, 1]], y_test),
    epochs=10, batch_size=64, verbose=1
)

Epoch 1/10


1250/1250 [==============================] - 101s 80ms/step - loss: 1.4646 - mse: 1.4646 - val_loss: 1.0936 - val_mse: 1.0936
Epoch 2/10
1250/1250 [==============================] - 93s 74ms/step - loss: 0.6279 - mse: 0.6279 - val_loss: 1.2007 - val_mse: 1.2007
Epoch 3/10
1250/1250 [==============================] - 101s 81ms/step - loss: 0.3831 - mse: 0.3831 - val_loss: 1.2804 - val_mse: 1.2804
Epoch 4/10
1250/1250 [==============================] - 89s 71ms/step - loss: 0.2174 - mse: 0.2174 - val_loss: 1.2349 - val_mse: 1.2349
Epoch 5/10
1250/1250 [==============================] - 88s 70ms/step - loss: 0.1253 - mse: 0.1253 - val_loss: 1.2324 - val_mse: 1.2324
Epoch 6/10
1250/1250 [==============================] - 89s 71ms/step - loss: 0.0952 - mse: 0.0952 - val_loss: 1.2542 - val_mse: 1.2542
Epoch 7/10
1250/1250 [==============================] - 97s 77ms/step - loss: 0.0759 - mse: 0.0759 - val_loss: 1.2608 - val_mse: 1.2608
Epoch 8/10
1250/1250 [==============================] - 9

# Model Prediction and Evaluation (RMSE)

In [12]:
y_pred = model.predict([x_test[:, 0], x_test[:, 1]])
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error (RMSE): {rmse}')

625/625 [==============================] - 1s 1ms/step
Root Mean Squared Error (RMSE): 1.1292402903878196
